<a href="https://colab.research.google.com/github/Ananyas-7/ai-dyslexia-tool/blob/main/method2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets sentencepiece accelerate jiwer tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.2 MB/s eta 0:00:00


In [2]:
import random, math, os
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
import pandas as pd
import torch
from tqdm import tqdm
from jiwer import wer, cer
device = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", device)


DEVICE: cuda


In [3]:
model_name = "t5-base"
num_synthetic_per_line = 3
target_dataset_size = 10000
max_length = 128
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
num_train_epochs = 8
learning_rate = 3e-4
output_dir = "./ocr_corrector_t5_base"

In [4]:
print("Loading Teklia/IAM-line (this may take a little)...")
ds = load_dataset("Teklia/IAM-line", split="train")
lines = [ex["text"].strip() for ex in ds if ex["text"] and len(ex["text"].split()) > 1]

# keep only reasonably short lines (avoid very long lines)
lines = [l for l in lines if len(l) < 120]
print(f"Available clean lines: {len(lines)}")

# if not enough lines, fall back to wikitext
if len(lines) < 200:
    print("Too few lines found in Teklia/IAM-line; falling back to wikitext-2-raw-v1")
    ds2 = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
    lines = [l["text"].strip() for l in ds2 if l["text"] and len(l["text"].split()) > 3]
    lines = [l for l in lines if len(l) < 120]
    print("Fallback lines:", len(lines))


Loading Teklia/IAM-line (this may take a little)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

data/validation.parquet:   0%|          | 0.00/24.7M [00:00<?, ?B/s]

data/test.parquet:   0%|          | 0.00/73.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6482 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/976 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2915 [00:00<?, ? examples/s]

Available clean lines: 6480


In [5]:
confusion_map = {
    "m": ["rn"],
    "rn": ["m"],
    "o": ["0"],
    "0": ["o"],
    "i": ["l"],
    "l": ["i"],
    "u": ["v"],
    "v": ["u"],
    "c": ["e"],
    "e": ["c"],
    "s": ["5"],
    "5": ["s"],
    "g": ["q"],
    "q": ["g"],
    "t": ["f"],   # visual-ish swaps
    "h": ["b"]    # sometimes mistakes
}

vowels = "aeiou"

def inject_realistic_noise(text, prob_confuse=0.12, prob_delete=0.03, prob_dup=0.03, prob_space_err=0.06):
    # apply character-level visual confusions + small deletions/duplications + spacing noise
    out = []
    i = 0
    while i < len(text):
        ch = text[i]
        # keep letters and punctuation; treat groups for 'rn' confusion
        two = text[i:i+2]
        if random.random() < prob_confuse and two in confusion_map:
            out.append(random.choice(confusion_map[two]))
            i += 2
            continue
        if random.random() < prob_confuse and ch in confusion_map:
            out.append(random.choice(confusion_map[ch]))
            i += 1
            continue
        r = random.random()
        if r < prob_delete:
            # delete char (but avoid deleting spaces always)
            if ch != " ":
                i += 1
                continue
        elif r < prob_delete + prob_dup:
            out.append(ch*2)
        else:
            out.append(ch)
        i += 1
    noisy = "".join(out)
    # spacing errors: remove spaces or insert extra spaces randomly
    if random.random() < prob_space_err:
        # remove some spaces
        noisy = noisy.replace(" ", "", 1)
    if random.random() < prob_space_err:
        # insert spurious space between characters sometimes
        idx = random.randint(0, max(0, len(noisy)-2))
        noisy = noisy[:idx] + " " + noisy[idx:]
    # small chance to swap neighboring characters (transposition)
    if len(noisy) > 3 and random.random() < 0.05:
        idx = random.randint(0, len(noisy)-2)
        nl = list(noisy)
        nl[idx], nl[idx+1] = nl[idx+1], nl[idx]
        noisy = "".join(nl)
    return noisy

# quick sanity check
for sample in ["This is a test sentence", "Machine learning for OCR is fun", "Dyslexic handwriting is hard to read"]:
    print("orig:", sample, "-> noisy:", inject_realistic_noise(sample))

orig: This is a test sentence -> noisy: Tis is a tcst sentenc
orig: Machine learning for OCR is fun -> noisy: Maehine learning for OCR i5 fun
orig: Dyslexic handwriting is hard to read -> noisy: D5iexic hadwritlngg is hard f0 read


In [6]:
print("Building synthetic dataset...")
pairs = []
i = 0
pbar = tqdm(total=target_dataset_size)
while len(pairs) < target_dataset_size:
    line = random.choice(lines)
    noisy = inject_realistic_noise(line)
    # avoid empty or identical
    if len(noisy.strip()) == 0 or noisy.strip() == line.strip():
        continue
    pairs.append({"noisy": noisy, "clean": line})
    pbar.update(1)
pbar.close()
print("Total pairs:", len(pairs))

# Convert to HuggingFace Dataset and split
df = pd.DataFrame(pairs)
dataset = Dataset.from_pandas(df)
split = dataset.train_test_split(test_size=0.05, seed=42)
train_ds = split["train"]
eval_ds = split["test"]
print("Train size:", len(train_ds), "Eval size:", len(eval_ds))

Building synthetic dataset...


100%|██████████| 10000/10000 [00:00<00:00, 10029.47it/s]


Total pairs: 10000
Train size: 9500 Eval size: 500


In [8]:
print("Loading model and tokenizer:", model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

Loading model and tokenizer: t5-base


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
def preprocess(batch):
    inputs = tokenizer(batch["noisy"], padding="max_length", truncation=True, max_length=max_length)
    labels = tokenizer(batch["clean"], padding="max_length", truncation=True, max_length=max_length)
    # replace padding token id's of the labels by -100 so they are ignored by the loss
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    inputs["labels"] = labels["input_ids"]
    return inputs

train_tokenized = train_ds.map(preprocess, batched=True, remove_columns=["noisy","clean"])
eval_tokenized = eval_ds.map(preprocess, batched=True, remove_columns=["noisy","clean"])

print("Tokenized columns:", train_tokenized.column_names)

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenized columns: ['input_ids', 'attention_mask', 'labels']


In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    logging_strategy="steps",   # force step-based logs
    logging_dir="./logs",       # where logs will be written
    report_to="tensorboard",    # enables TensorBoard + keeps progress bar
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    save_total_limit=3,
    fp16=True if device=="cuda" else False,
    remove_unused_columns=False,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-374613428.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
print("Starting training...")
trainer.train()
print("Training finished. Saving model...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

Starting training...


Step,Training Loss,Validation Loss
500,0.711400,0.515892
1000,0.396100,0.400964
1500,0.219200,0.356620
2000,0.130800,0.333383
2500,0.078400,0.328782
3000,0.081100,0.304828
3500,0.058700,0.308453
4000,0.051300,0.311980
4500,0.053900,0.311980


Training finished. Saving model...


('./ocr_corrector_t5_base/tokenizer_config.json',
 './ocr_corrector_t5_base/special_tokens_map.json',
 './ocr_corrector_t5_base/spiece.model',
 './ocr_corrector_t5_base/added_tokens.json',
 './ocr_corrector_t5_base/tokenizer.json')

In [12]:
print("Running inference on eval set and computing WER/CER...")

def model_correct(text):
    # generate corrected text for a noisy input
    enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).to(device)
    out = model.generate(**enc, max_length=max_length, num_beams=4)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# pick a small sample to show results
sample_eval = eval_ds.select(range(min(50, len(eval_ds))))
rows = []
wer_list, cer_list = [], []

for ex in tqdm(sample_eval):
    noisy = ex["noisy"]
    clean = ex["clean"]
    pred = model_correct(noisy)
    rows.append({"noisy": noisy, "clean": clean, "pred": pred})
    try:
        wer_list.append(wer(clean, pred))
        cer_list.append(cer(clean, pred))
    except Exception:
        pass

results_df = pd.DataFrame(rows)
print(f"Avg WER (sample): {sum(wer_list)/len(wer_list):.3f}  Avg CER (sample): {sum(cer_list)/len(cer_list):.3f}")
display(results_df.head(15))

Running inference on eval set and computing WER/CER...


100%|██████████| 50/50 [00:20<00:00,  2.41it/s]

Avg WER (sample): 0.087  Avg CER (sample): 0.052


,noisy,clean,pred
0,"Mr. Iain Maccleod , thee Coionlal Secretary d...","Mr. Iain Macleod , the Colonial Secretary , de...","Mr. Iain Macleod , the Colonial Secretary denied"
1,programme fo the needs of oider employees,programme to the needs of older employees,programme to the needs of older employees
2,of angl irons as shown in Fiig. s . Whhen dete...,of angle irons as shown in Fig. 5 . When deter...,of angle irons as shown in Table . When determ...
3,"create a certtaln rnount of mess ,, it l5","create a certain amount of mess , it is","create a certain amount of mess , it is"
4,"Oniy Mr. Luca5's action5 , trefore , aro5c","Only Mr. Lucas's actions , therefore , arose","Only Mr. Lucas's actions , therefore , arose"
5,Lobby of the Houseof C0mrnons last,Lobby of the House of Commons last,Lobby of the House of Commons last
6,spot. JJoyce Egginton cables : PPresldent,spot . Joyce Egginton cables : President,spot . Joyce Egginton cables : President
7,"tions . Tbc Sennate Banking Comrnittec , wbich","tions . The Senate Banking Committee , which","tions . The Senate Banking Committee , which"
8,tax relieff for a jacket used,tax relief for a jacket used,tax relief for a jacket used
9,"5ystern a tril , addiingg tbat lt was beinq","system a trial , adding that it was being","sense a tril , adding that it was being"


In [13]:
results_df.to_csv("correction_results_sample.csv", index=False)
print("Saved correction_results_sample.csv and model at", output_dir)

Saved correction_results_sample.csv and model at ./ocr_corrector_t5_base
